To load dataset from kaggle to google colab, We have to first create our api token in our kaggle account, download its json file and then upload that file to colab. That api token will allow us to load dataset from kaggle to google colab. Below command is used to copy uploaded token file to kaggle directory on colab.

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

By running the below code, The first line will start downloading the dataset in .zip format. The second line is to extract the dataset from zip file.

In [ ]:
!kaggle datasets download -d alxmamaev/flowers-recognition
!unzip -q flowers-recognition.zip -d flower-dataset

In Below code box, we imported all the necessary libraries or modules that we will use in code later.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torchvision.models as models

Below is the code of our model. Since we have a very small dataset, we cannot achieve good accuracy with this dataset. So to achieve a good accuracy i had to apply some techniques to deal with the problem of having small dataset. Some of the techniques are Data augmentation, Transfer learning with resnet50 architecture, regularization etc. Overall accuracy that i achieved with all of the techniques mentioned is **90%**.

In [ ]:
# Define data transformations
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomRotation(degrees=30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset
dataset = ImageFolder(root='flower-dataset/flowers', transform=transform)

# Define the sizes of train and test splits
train_size = int(0.8 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # Remaining 20% for testing

# Divide the dataset into train and test splits
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


# Create data loaders for train and test sets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load a pre-trained ResNet-18 model and modify it
resnet_model = models.resnet50(pretrained=True)

num_classes = len(dataset.classes)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=0.0001)

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_model.to(device)

# Train the model
num_epochs = 10

for epoch in range(num_epochs):
    resnet_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate the model
resnet_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")
